In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import modules.standard as standard
import modules.plotGraph as plotGraph
import modules.centrality as centrality
import math
import h5py

In [2]:
def create_matrixs(n, seed= 11, maxLen= 100):#n from 10 to 100
    m= int(math.log(n))
    random.seed(seed)
    graphSD= standard.generate_random_standard_graph(n, m)
    graphWS= nx.watts_strogatz_graph(n, m*2, 0.1)
    graphER= nx.dense_gnm_random_graph(n, m*n)
    graphBA= nx.barabasi_albert_graph(n, m)
    graphs= [graphSD, graphWS, graphER, graphBA]
    myEmbdeddingNps= []
    centralitysNps= []
    for graph in graphs:
        idS, exclude=standard.compress_left_order(nx.to_numpy_array(graph))
        standardNP= standard.standardize(graph, traversal=list(idS)+list(exclude))# a kind of embedding,  150, 
        standardGraph= nx.Graph(standardNP)
        myEmbdeddingNp=standard.power_m_embedding(standardNP, Precision= 5, length= 20, m= 0.5)
        eigenvectorCentralityNp= centrality.eigenvector_centrality_embedding(standardGraph)
        closenessCentralityNp= centrality.closeness_centrality_embedding(standardGraph)
        degreeCentralityNp= centrality.degree_centrality_embedding(standardGraph)
        loadCentralityNp= centrality.load_centrality_embedding(standardGraph)
        betweennessCentralityNp= centrality.betweenness_centrality_embedding(standardGraph)
        centralitys=[eigenvectorCentralityNp, closenessCentralityNp, degreeCentralityNp, loadCentralityNp, betweennessCentralityNp]
        centralitysNp= np.stack(centralitys)
        centralitysNpMax= np.zeros((5, maxLen))
        centralitysNpMax[:, 0: n]= centralitysNp
        myEmbdeddingNps.append(myEmbdeddingNp)
        centralitysNps.append(centralitysNpMax)
    return myEmbdeddingNps, centralitysNps


In [3]:
myEmbdeddingNpss= []
centralitysNpss= []
lenData= 2500
count= 1
maxLen= 100
for i in tqdm(range(10, 100)):
    for j in range(50):
        seed= i*50+j
        EmbdeddingNps, mycentralitysNps= create_matrixs(i, seed, maxLen)
        myEmbdeddingNpss+= EmbdeddingNps
        centralitysNpss+= mycentralitysNps
        if len(myEmbdeddingNpss)>=lenData:
            with h5py.File(f'datas/data{count}.h5', 'w') as f:#2500 matrices one file
                for k in range(lenData):
                    # Create a dataset from the NumPy array
                    f.create_dataset(f'M{k}', data= myEmbdeddingNpss[k])
                    f.create_dataset(f'C1{k}', data= centralitysNpss[k])
            print(f"save file: datas/data{count}.h5")
            count+=1
            myEmbdeddingNpss.clear()
            centralitysNpss.clear() 

#5*90*10

 13%|█▎        | 12/90 [00:06<00:46,  1.68it/s]

save file: datas/data1.h5


 28%|██▊       | 25/90 [00:20<01:24,  1.29s/it]

save file: datas/data2.h5


 41%|████      | 37/90 [00:41<01:45,  2.00s/it]

...

 83%|████████▎ | 75/90 [03:23<01:32,  6.19s/it]

save file: datas/data6.h5


 97%|█████████▋| 87/90 [04:52<00:23,  7.81s/it]

save file: datas/data7.h5


100%|██████████| 90/90 [05:18<00:00,  3.53s/it]
